In [1]:
import sklearn
import lime
import lime.lime_tabular
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn import tree
import pandas as pd
from random import randint

import seaborn as sns
%matplotlib inline

np.random.seed(1)


C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Program Files\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Program Files\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Functions

In [2]:
def evaluate_model(model, train, labels_train, test, labels_test, verbose=True):
    acc_train = sklearn.metrics.accuracy_score(labels_train, model.predict(train))
    acc_test = sklearn.metrics.accuracy_score(labels_test, model.predict(test))
    auc_train = sklearn.metrics.roc_auc_score(labels_train, model.predict(train))
    auc_test = sklearn.metrics.roc_auc_score(labels_test, model.predict(test))
    if verbose:
        print("Train Accuracy: " + "{:.3f}".format(acc_train) + " - Train AUC: " + "{:.3f}".format(auc_train))
        print("Test  Accuracy: " + "{:.3f}".format(acc_test) + " - Test AUC: " + "{:.3f}".format(auc_test))
    return acc_train, auc_train, acc_test, auc_test

def obtain_explanations(explainer, model, X, feature_names):
    # Explaining all predictions
    explanations = np.zeros((X.shape[0], X.shape[1]))
    exp_scores = []
    for i in range(len(X)):
        exp = explainer.explain_instance(X[i], model.predict_proba, num_features=len(feature_names), top_labels=1)
        key_label = [*exp.as_map()][0]
        for feature_exp in exp.as_map()[key_label]:
            feature_idx = feature_exp[0]
            feature_val = feature_exp[1]
            explanations[i, feature_idx] = feature_val
        #if i % 100 == 0:
        exp_scores.append(exp.score)
            #print(i)
            
    return explanations, exp_scores

# Create dataset

In [4]:
data, labels = sklearn.datasets.make_classification(n_samples=5000, 
                                     n_features=20, 
                                     n_informative=20, 
                                     n_redundant=0, 
                                     n_repeated=0, 
                                     n_classes=2, 
                                     n_clusters_per_class=2, 
                                     weights=None, 
                                     flip_y=0.02, 
                                     class_sep=1.0, 
                                     hypercube=True, 
                                     shift=0.0, 
                                     scale=1.0, 
                                     shuffle=True, 
                                     random_state=1)

In [3]:
feature_names = ['feature_' + str(i) for i in range(data.shape[1])]
target_names = ['label_1']

# Split dataset
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80)
print('Num training samples: ' + str(len(train)))
print('Num test samples: ' + str(len(test)))

NameError: name 'data' is not defined

In [6]:
np.savetxt("train4.csv", train, delimiter=",")
np.savetxt("test4.csv", test, delimiter=",")
np.savetxt("labels_train4.csv", labels_train, delimiter=",")
np.savetxt("labels_test4.csv", labels_test, delimiter=",")

In [3]:
# Use this to load the csv instead of creating it
train = np.genfromtxt("train4.csv", delimiter=',')
test = np.genfromtxt("test4.csv", delimiter=',')
labels_train = np.genfromtxt("labels_train4.csv", delimiter=',')
labels_test = np.genfromtxt("labels_test4.csv", delimiter=',')

feature_names = ['feature_' + str(i) for i in range(train.shape[1])]
target_names = ['label_1']

# Logistic Regression

https://math.stackexchange.com/questions/889425/what-does-determinant-of-covariance-matrix-give

https://stats.stackexchange.com/questions/225434/a-measure-of-variance-from-the-covariance-matrix


In [8]:
features_to_use = [[0,1],
                  [0,1,2],
                  [0,1,2,3],
                  [0,1,2,3,4],
                  [0,1,2,3,4,5],
                  [0,1,2,3,4,5,6],
                  [0,1,2,3,4,5,6,7],
                  [0,1,2,3,4,5,6,7,8],
                  [0,1,2,3,4,5,6,7,8,9],
                  [0,1,2,3,4,5,6,7,8,9,10],
                  [0,1,2,3,4,5,6,7,8,9,10,11],
                  [0,1,2,3,4,5,6,7,8,9,10,11,12],
                  [0,1,2,3,4,5,6,7,8,9,10,11,12,13],
                  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
                  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
                  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
                  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
                  ]

l_repetitions = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ,2, 2]
results_lr = {}


for i in range(len(features_to_use)):
    n_features = len(features_to_use[i])
    n_reps = l_repetitions[i]
    print('Starting with ', n_features, ' features')
    results_lr[n_features] = {}
    results_lr[n_features]['trace'] = []
    results_lr[n_features]['trace_normalized'] = []
    results_lr[n_features]['determinant'] = []
    results_lr[n_features]['determinant_normalized'] = []
    results_lr[n_features]['auc_train'] = []
    results_lr[n_features]['auc_test'] = []
    results_lr[n_features]['used_features'] = []
    results_lr[n_features]['max_eigenvalue'] = []
    results_lr[n_features]['explanation_fit'] = []
    
    for j in range(n_reps):
        # Pick n_features randomly
        idx_features = features_to_use[i]
        f_names = list(np.array(feature_names)[idx_features])
        results_lr[n_features]['used_features'].append(idx_features)
        
        model = linear_model.LogisticRegression(C=1)
        model.fit(train[:,idx_features], labels_train)
        #print("Model intercept: ", model.intercept_)
        acc_train, auc_train, acc_test, auc_test = evaluate_model(model, train[:,idx_features], labels_train, test[:,idx_features], labels_test, verbose=False)
        results_lr[n_features]['auc_train'].append(acc_train)
        results_lr[n_features]['auc_test'].append(acc_test)

        # Create explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(train[:,idx_features], feature_names=f_names, class_names=target_names, discretize_continuous=True)

        # Explaining all samples
        explanations_test, exp_score = obtain_explanations(explainer, model, test[:,idx_features], f_names)
        np.savetxt("explanations_data/explanations_lr_" + str(n_features) + "_" + str(j)  + ".csv", explanations_test, delimiter=",")
        results_lr[n_features]['explanation_fit'].append(np.mean(exp_score))
        
        # Compute covariance matrix of Explanations datasets.
        cov_matrix_test = np.cov(explanations_test, rowvar=False)
        np.savetxt("results/cov_lr_" + str(n_features) + "_" + str(j)  + ".csv", cov_matrix_test, delimiter=",")

        # Trace of covariance matrix
        results_lr[n_features]['trace'].append(np.trace(cov_matrix_test))
        results_lr[n_features]['trace_normalized'].append(np.trace(cov_matrix_test)/cov_matrix_test.shape[0])
        results_lr[n_features]['determinant'].append(np.linalg.det(cov_matrix_test))
        results_lr[n_features]['determinant_normalized'].append(np.linalg.det(cov_matrix_test)/cov_matrix_test.shape[0])
        
        # max-eigenvalues
        eigen_val, eigen_vectors = np.linalg.eig(cov_matrix_test)
        results_lr[n_features]['max_eigenvalue'].append(np.max(eigen_val))

    print(results_lr[n_features]['used_features'])
    print('auc_train mean: ', np.mean(results_lr[n_features]['auc_train']))
    print('auc_test mean: ', np.mean(results_lr[n_features]['auc_test']))
    print('trace mean: ', np.mean(results_lr[n_features]['trace']))
    print('trace_normalized mean: ', np.mean(results_lr[n_features]['trace_normalized']))
    print('median trace_normalized: ', np.median(results_lr[n_features]['trace_normalized']))
    print('mean max_eigenvalue: ', np.mean(results_lr[n_features]['max_eigenvalue']))
    print('explanation fit: ', np.mean(results_lr[n_features]['explanation_fit']))
    print('-----------------------------')


Starting with  2  features
[[0, 1], [0, 1]]
auc_train mean:  0.56425
auc_test mean:  0.593
trace mean:  0.004900036894587331
trace_normalized mean:  0.0024500184472936656
median trace_normalized:  0.0024500184472936656
mean max_eigenvalue:  0.0045611539263340235
explanation fit:  0.3376164902949421
-----------------------------
Starting with  3  features
[[0, 1, 2], [0, 1, 2]]
auc_train mean:  0.6075
auc_test mean:  0.615
trace mean:  0.02079442316701109
trace_normalized mean:  0.0069314743890036955
median trace_normalized:  0.0069314743890036955
mean max_eigenvalue:  0.015057045235209829
explanation fit:  0.3270525146981462
-----------------------------
Starting with  4  features
[[0, 1, 2, 3], [0, 1, 2, 3]]
auc_train mean:  0.64175
auc_test mean:  0.624
trace mean:  0.035251420118251095
trace_normalized mean:  0.008812855029562774
median trace_normalized:  0.008812855029562774
mean max_eigenvalue:  0.01614815154289502
explanation fit:  0.32108197699527374
----------------------------

In [9]:
# Create df and save csv
model_names = ['LR - ' + str(k) for k in results_lr.keys()]
mean_auc_train = [np.mean(results_lr[k]['auc_train']) for k in results_lr.keys()]
mean_auc_test = [np.mean(results_lr[k]['auc_test']) for k in results_lr.keys()]
mean_trace = [np.mean(results_lr[k]['trace']) for k in results_lr.keys()]
median_trace = [np.median(results_lr[k]['trace']) for k in results_lr.keys()]
mean_trace_norm = [np.mean(results_lr[k]['trace_normalized']) for k in results_lr.keys()]
mean_max_eig = [np.mean(results_lr[k]['max_eigenvalue']) for k in results_lr.keys()]
median_max_eig = [np.median(results_lr[k]['max_eigenvalue']) for k in results_lr.keys()]
mean_explanation_fit = [np.mean(results_lr[k]['explanation_fit']) for k in results_lr.keys()]
median_explanation_fit = [np.median(results_lr[k]['explanation_fit']) for k in results_lr.keys()]

df_results_lr = pd.DataFrame()
df_results_lr['model_name'] = model_names
df_results_lr['auc_train'] = mean_auc_train
df_results_lr['auc_test'] = mean_auc_test
df_results_lr['mean_trace'] = mean_trace
df_results_lr['median_trace'] = median_trace
df_results_lr['trace_norm'] = mean_trace_norm
df_results_lr['mean_max_eig'] = mean_max_eig
df_results_lr['median_max_eig'] = median_max_eig
df_results_lr['mean_explanation_fit'] = mean_explanation_fit
df_results_lr['median_explanation_fit'] = median_explanation_fit
df_results_lr.to_csv('df_results_lr.csv', index=False)

In [10]:
df_results_lr

,model_name,auc_train,auc_test,mean_trace,median_trace,trace_norm,mean_max_eig,median_max_eig,mean_explanation_fit,median_explanation_fit
0,LR - 2,0.56425,0.593,0.004900,0.004900,0.002450,0.004561,0.004561,0.337616,0.337616
1,LR - 3,0.60750,0.615,0.020794,0.020794,0.006931,0.015057,0.015057,0.327053,0.327053
2,LR - 4,0.64175,0.624,0.035251,0.035251,0.008813,0.016148,0.016148,0.321082,0.321082
3,LR - 5,0.64025,0.636,0.036666,0.036666,0.007333,0.015869,0.015869,0.315620,0.315620
4,LR - 6,0.66725,0.656,0.065554,0.065554,0.010926,0.025964,0.025964,0.309757,0.309757
5,LR - 7,0.68050,0.681,0.066736,0.066736,0.009534,0.023937,0.023937,0.308020,0.308020
6,LR - 8,0.69675,0.704,0.083687,0.083687,0.010461,0.029087,0.029087,0.301737,0.301737
7,LR - 9,0.70150,0.705,0.090173,0.090173,0.010019,0.030443,0.030443,0.299203,0.299203
8,LR - 10,0.70175,0.704,0.091179,0.091179,0.009118,0.031096,0.031096,0.297994,0.297994
9,LR - 11,0.70025,0.707,0.091471,0.091471,0.008316,0.031232,0.031232,0.297140,0.297140


In [ ]:
import numpy as np
import matplotlib.pyplot as plt



# plt.subplot(2, 1, 1)
plt.plot([2, 3, 4, 5, 6, 7, 8, 9, 10], [0.0069, 0.00628, 0.018615617, 0.015079936, 0.012565618, 0.012115437, 0.012049166, 0.011108142, 0.011183237], 'o-')
plt.title('Logistic Regression - Opacity Score')
plt.ylabel('Mean normalized trace cov. explanations matrix')
plt.xlabel('Num. variables')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


num_vars = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
score = df_results_lr['mean_explanation_fit']

# plt.subplot(2, 1, 1)
plt.plot(num_vars, score, 'o-')
plt.title('Logistic Regression - Opacity Score')
plt.ylabel('Mean trace cov. explanations matrix')
plt.xlabel('Num. variables')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


num_vars = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
score = [0.0138, 0.01886, 0.0744764, 0.075445792, 0.075410861, 0.084822101, 0.096375579, 0.099940218, 0.111810713, 0.1189631, 0.11911497, 0.120912857, 0.120802075, 0.133137306, 0.138690919, 0.138632272, 0.141359093]

# plt.subplot(2, 1, 1)
plt.plot(num_vars, score, 'o-')
plt.title('Logistic Regression - Opacity Score')
plt.ylabel('Mean trace cov. explanations matrix')
plt.xlabel('Num. variables')

# Decision Trees

In [42]:
l_max_depth = [3]#[2, 4, 6, 8, 10, 12, 14, 16, 18]
l_repetitions = [1, 1, 1, 1, 1, 1, 1, 1, 1]
results_dt = {}


for i in range(len(l_max_depth)):
    max_depth = l_max_depth[i]
    n_reps = l_repetitions[i]
    print('Starting with ', max_depth, ' max_depth')
    results_dt[max_depth] = {}
    results_dt[max_depth]['trace'] = []
    results_dt[max_depth]['trace_normalized'] = []
    results_dt[max_depth]['determinant'] = []
    results_dt[max_depth]['determinant_normalized'] = []
    results_dt[max_depth]['auc_train'] = []
    results_dt[max_depth]['auc_test'] = []
    results_dt[max_depth]['used_features'] = []
    results_dt[max_depth]['max_depth'] = []
    results_dt[max_depth]['max_eigenvalue'] = []
    results_dt[max_depth]['explanation_fit'] = []
    
    for j in range(n_reps):
        print('Iteration ', j, ' of ', n_reps)
        
        model = DecisionTreeClassifier(max_depth=max_depth)
        model.fit(train, labels_train)
        results_dt[max_depth]['max_depth'].append(model.tree_.max_depth) 
        acc_train, auc_train, acc_test, auc_test = evaluate_model(model, train, labels_train, test, labels_test, verbose=True)
        results_dt[max_depth]['auc_train'].append(acc_train)
        results_dt[max_depth]['auc_test'].append(acc_test)

        # Create explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=feature_names, class_names=target_names, discretize_continuous=True)

        # Explaining all samples
        explanations_test, exp_score = obtain_explanations(explainer, model, test, feature_names)
        np.savetxt("explanations_data/explanations_dt_" + str(max_depth) + "_" + str(j)  + ".csv", explanations_test, delimiter=",")
        results_dt[max_depth]['explanation_fit'].append(np.mean(exp_score))

        # Compute covariance matrix of Explanations datasets.
        cov_matrix_test = np.cov(explanations_test, rowvar=False)
        np.savetxt("results/cov_dt_" + str(max_depth) + "_" + str(j)  + ".csv", cov_matrix_test, delimiter=",")      

        # Trace of covariance matrix
        results_dt[max_depth]['trace'].append(np.trace(cov_matrix_test))
        results_dt[max_depth]['trace_normalized'].append(np.trace(cov_matrix_test)/cov_matrix_test.shape[0])
        results_dt[max_depth]['determinant'].append(np.linalg.det(cov_matrix_test))
        results_dt[max_depth]['determinant_normalized'].append(np.linalg.det(cov_matrix_test)/cov_matrix_test.shape[0])
        
        # Eigenvalues
        eigen_val, eigen_vectors = np.linalg.eig(cov_matrix_test)
        results_dt[max_depth]['max_eigenvalue'].append(np.max(eigen_val))
        
    print(results_dt[max_depth]['max_depth'])
    print('auc_train mean: ', np.mean(results_dt[max_depth]['auc_train']))
    print('auc_test mean: ', np.mean(results_dt[max_depth]['auc_test']))
    print('trace mean: ', np.mean(results_dt[max_depth]['trace']))
    print('trace_normalized mean: ', np.mean(results_dt[max_depth]['trace_normalized']))
    print('median trace_normalized: ', np.median(results_dt[max_depth]['trace_normalized']))
    print('mean max_eigenvalue: ', np.mean(results_dt[max_depth]['max_eigenvalue']))
    print('explanation fit: ', np.mean(results_dt[max_depth]['explanation_fit']))


Starting with  3  max_depth
Iteration  0  of  1
Train Accuracy: 0.769 - Train AUC: 0.769
Test  Accuracy: 0.753 - Test AUC: 0.751
[3]
auc_train mean:  0.769
auc_test mean:  0.753
trace mean:  0.06102699287778197
trace_normalized mean:  0.0030513496438890984
median trace_normalized:  0.0030513496438890984
mean max_eigenvalue:  0.028333855042091433
explanation fit:  0.18661832435582576


In [43]:
# Create df and save df
model_names = ['DT - ' + str(k) for k in l_max_depth]
mean_auc_train = [np.mean(results_dt[k]['auc_train']) for k in l_max_depth]
mean_auc_test = [np.mean(results_dt[k]['auc_test']) for k in l_max_depth]
mean_trace = [np.mean(results_dt[k]['trace']) for k in l_max_depth]
median_trace = [np.median(results_dt[k]['trace']) for k in l_max_depth]
mean_trace_norm = [np.mean(results_dt[k]['trace_normalized']) for k in l_max_depth]
mean_max_eig = [np.mean(results_dt[k]['max_eigenvalue']) for k in l_max_depth]
median_max_eig = [np.median(results_dt[k]['max_eigenvalue']) for k in l_max_depth]
mean_explanation_fit = [np.mean(results_dt[k]['explanation_fit']) for k in l_max_depth]
median_explanation_fit = [np.median(results_dt[k]['explanation_fit']) for k in l_max_depth]

df_results_dt = pd.DataFrame()
df_results_dt['max_depth'] = l_max_depth
df_results_dt['model_name'] = model_names
df_results_dt['auc_train'] = mean_auc_train
df_results_dt['auc_test'] = mean_auc_test
df_results_dt['mean_trace'] = mean_trace
df_results_dt['median_trace'] = median_trace
df_results_dt['trace_norm'] = mean_trace_norm
df_results_dt['mean_max_eig'] = mean_max_eig
df_results_dt['median_max_eig'] = median_max_eig
df_results_dt['mean_explanation_fit'] = mean_explanation_fit
df_results_dt['median_explanation_fit'] = median_explanation_fit
df_results_dt.to_csv('df_results_dt_maxdepth3.csv', index=False)

In [44]:
df_results_dt

,max_depth,model_name,auc_train,auc_test,mean_trace,median_trace,trace_norm,mean_max_eig,median_max_eig,mean_explanation_fit,median_explanation_fit
0,3,DT - 3,0.769,0.753,0.061027,0.061027,0.003051,0.028334,0.028334,0.186618,0.186618


In [20]:
df_results_dt

,max_depth,model_name,auc_train,auc_test,mean_trace,median_trace,trace_norm,mean_max_eig,median_max_eig,mean_explanation_fit,median_explanation_fit
0,2,DT - 2,0.72825,0.723,0.042311,0.042311,0.002116,0.027455,0.027455,0.189732,0.189732
1,4,DT - 4,0.79750,0.757,0.067557,0.067557,0.003378,0.026865,0.026865,0.159078,0.159078
2,6,DT - 6,0.87875,0.821,0.074078,0.074078,0.003704,0.024192,0.024192,0.112860,0.112860
3,8,DT - 8,0.92975,0.827,0.089358,0.089358,0.004468,0.026204,0.026204,0.099851,0.099851
4,10,DT - 10,0.96775,0.838,0.087945,0.087945,0.004397,0.022747,0.022747,0.082671,0.082671
5,12,DT - 12,0.98800,0.824,0.092019,0.092019,0.004601,0.024383,0.024383,0.079674,0.079674
6,14,DT - 14,0.99425,0.836,0.088827,0.088827,0.004441,0.023500,0.023500,0.075596,0.075596
7,16,DT - 16,0.99650,0.824,0.093545,0.093545,0.004677,0.025176,0.025176,0.078727,0.078727
8,18,DT - 18,0.99925,0.821,0.093409,0.093409,0.004670,0.023235,0.023235,0.077845,0.077845


In [ ]:
import numpy as np
import matplotlib.pyplot as plt



# plt.subplot(2, 1, 1)
plt.plot([2, 4, 6, 8, 10, 12, 14, 16, 17], [0.002141623, 0.0028, 0.003019607, 0.00323, 0.003348, 0.003374, 0.003318705, 0.003386327, 0.003397824] , 'o-')
plt.title('Decision Tree - Opacity Score')
plt.ylabel('Median normalized trace cov. explanations matrix')
plt.xlabel('Max depth.')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# plt.subplot(2, 1, 1)
plt.plot([2, 4, 6, 8, 10, 12, 14, 16, 17], [0.042832462, 0.0574, 0.06039213, 0.06474, 0.06696, 0.06749, 0.066374093, 0.067726538, 0.067956477] , 'o-')
plt.title('Decision Tree - Opacity Score')
plt.ylabel('Trace cov. explanations matrix')
plt.xlabel('Max depth.')

In [ ]:
# Decision Trees - AUC

import numpy as np
import matplotlib.pyplot as plt

dt_auc_train = [0.726875, 0.765, 0.8515625, 0.9259375, 0.9653, 0.9825, 0.9946875, 0.9990625, 1]
df_auc_test = [0.69875, 0.7375, 0.78575, 0.80425, 0.80775, 0.8015, 0.8065, 0.8075, 0.8059375]

# plt.subplot(2, 1, 1)
plt.plot([2, 4, 6, 8, 10, 12, 14, 16, 17], dt_auc_train , 'o-')
plt.plot([2, 4, 6, 8, 10, 12, 14, 16, 17], df_auc_test, 'o-')
plt.title('Decision Tree - Opacity Score')
plt.ylabel('AUC')
plt.xlabel('Max depth.')
plt.legend(['auc_train', 'auc_test'], loc='upper left')

# Random Forest

In [ ]:
max_depth = 8
l_n_estimators = [2, 3, 5, 8, 12, 16, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100]
n_executions = 5
results_rf2 = {}


for n_estimators in l_n_estimators:
    print('Computing for ', n_estimators)
    results_rf2[n_estimators] = {}
    results_rf2[n_estimators]['auc_train'] = []
    results_rf2[n_estimators]['auc_test'] = []
    results_rf2[n_estimators]['trace'] = []
    results_rf2[n_estimators]['max_eigenvalue'] = []
    results_rf2[n_estimators]['explanation_fit'] = []
    
    
    for j in range(n_executions):
        
        model = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, 
                                                max_depth=max_depth, 
                                                min_samples_split=2,
                                                min_samples_leaf=1
                                                       )
        model.fit(train, labels_train)

        acc_train, auc_train, acc_test, auc_test = evaluate_model(model, train, labels_train, test, labels_test, verbose=False)

        # Create explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=feature_names, class_names=target_names, discretize_continuous=True)

        # Explaining all samples
        explanations_test, exp_score = obtain_explanations(explainer, model, test, feature_names)
        np.savetxt("explanations_data/explanations_rf_" + str(n_estimators) + "_" + str(j)  + ".csv", explanations_test, delimiter=",")      

        # Compute covariance matrix of Explanations datasets.
        cov_matrix_test = np.cov(explanations_test, rowvar=False)
        np.savetxt("results/cov_rf_" + str(n_estimators) + "_" + str(j)  + ".csv", cov_matrix_test, delimiter=",")

        # Compute eigenvalues and eigenvectors of covariance matrix
        eigen_val, eigen_vectors = np.linalg.eig(cov_matrix_test)

        # Add results
        results_rf2[n_estimators]['auc_train'].append(auc_train)
        results_rf2[n_estimators]['auc_test'].append(auc_test)
        results_rf2[n_estimators]['trace'].append(np.trace(cov_matrix_test))
        results_rf2[n_estimators]['max_eigenvalue'].append(np.max(eigen_val))
        results_rf2[n_estimators]['explanation_fit'].append(np.mean(exp_score))
        

    print('auc_train mean: ', np.mean(results_rf2[n_estimators]['auc_train']))
    print('auc_test mean: ', np.mean(results_rf2[n_estimators]['auc_test']))
    print('trace mean: ', np.mean(results_rf2[n_estimators]['trace']))
    print('mean max_eigenvalue: ', np.mean(results_rf2[n_estimators]['max_eigenvalue']))
    print('explanation fit: ', np.mean(results_rf2[n_estimators]['explanation_fit']))
    print('-----------------------------')
        
        

Computing for  2
auc_train mean:  0.8917460679370869
auc_test mean:  0.797702055465971
trace mean:  0.06531514905335616
mean max_eigenvalue:  0.022749146256767615
explanation fit:  0.12535610449225695
-----------------------------
Computing for  3
auc_train mean:  0.9160028560456966
auc_test mean:  0.8349265161912459
trace mean:  0.05599724239080206
mean max_eigenvalue:  0.018559037993238547
explanation fit:  0.14096270704947464
-----------------------------
Computing for  5
auc_train mean:  0.9406128498055969
auc_test mean:  0.8630436261348677
trace mean:  0.05474415732377598
mean max_eigenvalue:  0.019196114911305618
explanation fit:  0.17791732237674499
-----------------------------
Computing for  8
auc_train mean:  0.9570127122033952
auc_test mean:  0.8886159115553438
trace mean:  0.05258099713100498
mean max_eigenvalue:  0.015668283498688852
explanation fit:  0.1991204714082555
-----------------------------
Computing for  12
auc_train mean:  0.9641614265828252
auc_test mean:  0.89

In [22]:
# Create df and save df
model_names = ['RF - 3 max_depth - ' + str(k) + ' trees' for k in l_n_estimators]
mean_auc_train = [np.mean(results_rf2[k]['auc_train']) for k in l_n_estimators]
mean_auc_test = [np.mean(results_rf2[k]['auc_test']) for k in l_n_estimators]
mean_trace = [np.mean(results_rf2[k]['trace']) for k in l_n_estimators]
median_trace = [np.median(results_rf2[k]['trace']) for k in l_n_estimators]
mean_max_eig = [np.mean(results_rf2[k]['max_eigenvalue']) for k in l_n_estimators]
median_max_eig = [np.median(results_rf2[k]['max_eigenvalue']) for k in l_n_estimators]
mean_explanation_fit = [np.mean(results_rf2[k]['explanation_fit']) for k in l_n_estimators]
median_explanation_fit = [np.median(results_rf2[k]['explanation_fit']) for k in l_n_estimators]

df_results_rf2 = pd.DataFrame()
df_results_rf2['max_depth'] = l_n_estimators
df_results_rf2['model_name'] = model_names
df_results_rf2['auc_train'] = mean_auc_train
df_results_rf2['auc_test'] = mean_auc_test
df_results_rf2['mean_trace'] = mean_trace
df_results_rf2['median_trace'] = median_trace
df_results_rf2['mean_max_eig'] = mean_max_eig
df_results_rf2['median_max_eig'] = median_max_eig
df_results_rf2['mean_explanation_fit'] = mean_explanation_fit
df_results_rf2['median_explanation_fit'] = median_explanation_fit
df_results_rf2.to_csv('df_results_rf_max_depth8.csv', index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# plt.subplot(2, 1, 1)
rt_n_estimators = [2, 3, 4, 5, 6, 7, 8, 9, 10]
rt_opacity_score_1 = [0.023719672, 0.020162563, 0.013179423, 0.018304094, 0.017342572, 0.015894145, 0.017758619, 0.016590274, 0.015305451]
rt_opacity_score_2 = [0.040151616, 0.046891136, 0.043122655, 0.041279204, 0.040748781, 0.040559727, 0.038958451, 0.045356865, 0.048994338]
plt.plot(rt_n_estimators, rt_opacity_score_1 , 'o-')
plt.plot(rt_n_estimators, rt_opacity_score_2 , 'o-')
plt.title('Random Forest - Opacity Score')
plt.ylabel('Trace cov. explanations matrix')
plt.xlabel('Number of trees')
plt.legend(['max_depth = 3', 'max_depth = 8'], loc='upper left')


# Neural networks

In [6]:
# standarize data (also try without scaling)
scaler = StandardScaler()
scaler = scaler.fit(train)
train_scaled = scaler.transform(train)
test_scaled = scaler.transform(test)

In [ ]:
#architectures = [(10,), (20,), (30,), (40,), (50,), (60,), (70,), (80,), (90,), (100,)]
architectures = [(100,), (100,100,), (100,100,100,), (100,100,100,100,), (100,100,100,100,100), 
                 (100,100,100,100,100,100), (100,100,100,100,100,100,100), (100,100,100,100,100,100,100,100)]
n_executions = 5
results_nn8 = {}

train_use = train
test_use = test

for architecture in architectures:
    print('Computing for ', architecture)
    results_nn8[architecture] = {}
    results_nn8[architecture]['auc_train'] = []
    results_nn8[architecture]['auc_test'] = []
    results_nn8[architecture]['trace'] = []
    results_nn8[architecture]['max_eigenvalue'] = []
    results_nn8[architecture]['explanation_fit'] = []
    
    
    for j in range(n_executions):
        
        model = MLPClassifier(hidden_layer_sizes=architecture, 
                  activation='logistic', 
                  solver='adam', 
                  alpha=0.0001, 
                  batch_size=128, 
                  learning_rate='constant', 
                  learning_rate_init=0.001,  
                  max_iter=200, 
                  shuffle=True, 
                  #random_state=1, 
                  tol=0.0001, 
                  verbose=False, 
                  momentum=0.9, # Momentum for gradient descent update. Should be between 0 and 1. Only used when solver=’sgd’.
                  nesterovs_momentum=True, 
                  early_stopping=False, 
                  validation_fraction=0.1, 
                  beta_1=0.9,
                  beta_2=0.999, 
                  epsilon=1e-08)
        
        model.fit(train_use, labels_train)

        acc_train, auc_train, acc_test, auc_test = evaluate_model(model, train_use, labels_train, test_use, labels_test, verbose=False)

        # Create explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(train_use, feature_names=feature_names, class_names=target_names, discretize_continuous=True)

        # Explaining all samples
        explanations_test, exp_score = obtain_explanations(explainer, model, test_use, feature_names)
        np.savetxt("explanations_data/explanations_nn8_" + str(architecture) + "_notscaling_tanh_" + str(j)  + ".csv", explanations_test, delimiter=",")      

        # Compute covariance matrix of Explanations datasets.
        cov_matrix_test = np.cov(explanations_test, rowvar=False)
        np.savetxt("results/cov_nn7_" + str(architecture) + "_tanh_" + str(j)  + ".csv", cov_matrix_test, delimiter=",")

        # Compute eigenvalues and eigenvectors of covariance matrix
        eigen_val, eigen_vectors = np.linalg.eig(cov_matrix_test)

        # Add results
        results_nn8[architecture]['auc_train'].append(auc_train)
        results_nn8[architecture]['auc_test'].append(auc_test)
        results_nn8[architecture]['trace'].append(np.trace(cov_matrix_test))
        results_nn8[architecture]['max_eigenvalue'].append(np.max(eigen_val))
        results_nn8[architecture]['explanation_fit'].append(np.mean(exp_score))
        

    print('auc_train mean: ', np.mean(results_nn8[architecture]['auc_train']))
    print('auc_test mean: ', np.mean(results_nn8[architecture]['auc_test']))
    print('trace mean: ', np.mean(results_nn8[architecture]['trace']))
    print('mean max_eigenvalue: ', np.mean(results_nn8[architecture]['max_eigenvalue']))
    print('explanation fit: ', np.mean(results_nn8[architecture]['explanation_fit']))
    print('-----------------------------')
        
        

Computing for  (100,)


C:\Program Files\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


auc_train mean:  0.996898750380006
auc_test mean:  0.9695181238721347
trace mean:  0.16525267687869888
mean max_eigenvalue:  0.044858476267710085
explanation fit:  0.19591269843705408
-----------------------------
Computing for  (100, 100)
auc_train mean:  1.0
auc_test mean:  0.9627663362929588
trace mean:  0.1562439745226606
mean max_eigenvalue:  0.04169596736821722
explanation fit:  0.15667930386267265
-----------------------------
Computing for  (100, 100, 100)


In [7]:
# Create df and save df
model_names = ['NN - ' + str(k) for k in architectures]
mean_auc_train = [np.mean(results_nn8[k]['auc_train']) for k in architectures]
mean_auc_test = [np.mean(results_nn8[k]['auc_test']) for k in architectures]
mean_trace = [np.mean(results_nn8[k]['trace']) for k in architectures]
median_trace = [np.median(results_nn8[k]['trace']) for k in architectures]
mean_max_eig = [np.mean(results_nn8[k]['max_eigenvalue']) for k in architectures]
median_max_eig = [np.median(results_nn8[k]['max_eigenvalue']) for k in architectures]
mean_explanation_fit = [np.mean(results_nn8[k]['explanation_fit']) for k in architectures]
median_explanation_fit = [np.median(results_nn8[k]['explanation_fit']) for k in architectures]

df_results_nn8 = pd.DataFrame()
#results_nn['layer_spec'] = model_names
df_results_nn8['model_name'] = model_names
df_results_nn8['auc_train'] = mean_auc_train
df_results_nn8['auc_test'] = mean_auc_test
df_results_nn8['mean_trace'] = mean_trace
df_results_nn8['median_trace'] = median_trace
df_results_nn8['mean_max_eig'] = mean_max_eig
df_results_nn8['median_max_eig'] = median_max_eig
df_results_nn8['mean_explanation_fit'] = mean_explanation_fit
df_results_nn8['median_explanation_fit'] = median_explanation_fit
df_results_nn8.to_csv('df_results_nn_N_layers_no_scaling_tanh.csv', index=False)